# Fino podešavanje Open AI modela

Ovaj bilježnik temelji se na trenutnim uputama iz [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentacije Open AI.

Fino podešavanje poboljšava performanse temeljnih modela za vašu aplikaciju ponovnim treniranjem s dodatnim podacima i kontekstom relevantnim za taj specifični slučaj upotrebe ili scenarij. Imajte na umu da tehnike inženjeringa upita poput _few shot learning_ i _retrieval augmented generation_ omogućuju vam da poboljšate zadani upit relevantnim podacima za poboljšanje kvalitete. Međutim, ti pristupi ograničeni su maksimalnom veličinom prozora tokena ciljanog temeljog modela.

Fino podešavanje zapravo znači ponovno treniranje samog modela s potrebnim podacima (što nam omogućuje korištenje mnogo više primjera nego što stane u maksimalni prozor tokena) - i implementaciju _prilagođene_ verzije modela koja više ne treba primjere tijekom izvođenja. To ne samo da poboljšava učinkovitost dizajna našeg upita (imamo veću fleksibilnost u korištenju prozora tokena za druge stvari), već potencijalno i smanjuje naše troškove (smanjenjem broja tokena koje trebamo poslati modelu tijekom izvođenja).

Fino podešavanje ima 4 koraka:
1. Pripremite podatke za treniranje i učitajte ih.
1. Pokrenite zadatak treniranja da biste dobili fino podešeni model.
1. Procijenite fino podešeni model i iterirajte za kvalitetu.
1. Implementirajte fino podešeni model za izvođenje kad ste zadovoljni.

Imajte na umu da ne podržavaju svi temeljni modeli fino podešavanje - [provjerite OpenAI dokumentaciju](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) za najnovije informacije. Također možete fino podesiti prethodno fino podešeni model. U ovom vodiču koristit ćemo `gpt-35-turbo` kao naš ciljani temeljni model za fino podešavanje.

---


### Korak 1.1: Pripremite svoj skup podataka

Izgradimo chatbot koji vam pomaže razumjeti periodni sustav elemenata odgovarajući na pitanja o elementu limerikom. U _ovom_ jednostavnom vodiču, samo ćemo stvoriti skup podataka za treniranje modela s nekoliko primjera odgovora koji pokazuju očekivani format podataka. U stvarnom slučaju korištenja, trebali biste stvoriti skup podataka s mnogo više primjera. Također biste mogli koristiti otvoreni skup podataka (za vašu domenu primjene) ako postoji, i preformatirati ga za upotrebu u finoj prilagodbi.

Budući da se fokusiramo na `gpt-35-turbo` i tražimo odgovor u jednom koraku (chat dovršetak), možemo stvoriti primjere koristeći [ovaj predloženi format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) koji odražava zahtjeve OpenAI chat dovršetka. Ako očekujete višekratni razgovor, koristili biste [format primjera za višekratni razgovor](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) koji uključuje parametar `weight` za označavanje koje poruke treba koristiti (ili ne) u procesu fino podešavanja.

Za naš vodič koristit ćemo jednostavniji format s jednim korakom. Podaci su u [jsonl formatu](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) s jednim zapisom po retku, svaki predstavljen kao JSON objekt. Donji isječak prikazuje 2 zapisa kao primjer - pogledajte [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) za puni skup primjera (10 primjera) koje ćemo koristiti za naš vodič o finoj prilagodbi. **Napomena:** Svaki zapis _mora_ biti definiran u jednom retku (ne razdvajati ga preko više redaka kao što je uobičajeno u formatiranom JSON-u)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

U stvarnom slučaju korištenja trebat će vam mnogo veći skup primjera za dobre rezultate - kompromis će biti između kvalitete odgovora i vremena/troškova za fino podešavanje. Koristimo mali skup kako bismo brzo završili fino podešavanje i ilustrirali proces. Pogledajte [ovaj primjer iz OpenAI Cookbook-a](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) za složeniji vodič o finoj prilagodbi.


---

### Korak 1.2 Učitajte svoj skup podataka

Učitajte podatke koristeći Files API [kako je opisano ovdje](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Imajte na umu da kako biste mogli pokrenuti ovaj kod, morate prvo napraviti sljedeće korake:
 - Instalirali `openai` Python paket (pazite da koristite verziju >=0.28.0 za najnovije značajke)
 - Postavili varijablu okoline `OPENAI_API_KEY` na svoj OpenAI API ključ
Za više informacija, pogledajte [Vodič za postavljanje](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) koji je osiguran za tečaj.

Sada pokrenite kod za stvaranje datoteke za učitavanje iz vaše lokalne JSONL datoteke.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Korak 2.1: Kreirajte zadatak fino podešavanja pomoću SDK-a


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Korak 2.2: Provjerite status zadatka

Evo nekoliko stvari koje možete učiniti s `client.fine_tuning.jobs` API-jem:
- `client.fine_tuning.jobs.list(limit=<n>)` - Prikaži posljednjih n zadataka fino podešavanja
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Dohvati detalje određenog zadatka fino podešavanja
- `client.fine_tuning.jobs.cancel(<job_id>)` - Otkaži zadatak fino podešavanja
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Prikaži do n događaja iz zadatka
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvi korak procesa je _validacija datoteke za treniranje_ kako bi se osiguralo da su podaci u ispravnom formatu.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Korak 2.3: Pratite događaje za praćenje napretka


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Korak 2.4: Pregled statusa u OpenAI nadzornoj ploči


Također možete provjeriti status posjetom OpenAI web stranici i istraživanjem odjeljka _Fine-tuning_ na platformi. To će vam pokazati status trenutnog zadatka, kao i omogućiti praćenje povijesti prethodnih izvršenja zadataka. Na ovom snimku zaslona možete vidjeti da je prethodno izvršenje bilo neuspješno, a drugo izvršenje uspješno. Za kontekst, to se dogodilo kada je prvo izvršenje koristilo JSON datoteku s nepravilno formatiranim zapisima - nakon ispravka, drugo izvršenje je uspješno završilo i model je postao dostupan za korištenje.

![Fine-tuning job status](../../../../../translated_images/hr/fine-tuned-model-status.563271727bf7bfba.webp)


Također možete pregledati poruke statusa i metrike pomicanjem prema dolje u vizualnoj nadzornoj ploči kao što je prikazano:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/hr/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/hr/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Korak 3.1: Dohvati ID i testiraj fino podešeni model u kodu


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Korak 3.2: Učitajte i testirajte fino podešeni model u Playgroundu

Sada možete testirati fino podešeni model na dva načina. Prvo, možete posjetiti Playground i koristiti padajući izbornik Models za odabir vašeg novopodešenog modela s popisa opcija. Druga opcija je korištenje opcije "Playground" prikazane u panelu Fine-tuning (vidi snimku zaslona gore) koja pokreće sljedeći _usporedni_ prikaz koji prikazuje verzije osnovnog i fino podešenog modela jedna pored druge za brzu evaluaciju.

![Fine-tuning job status](../../../../../translated_images/hr/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Jednostavno ispunite kontekst sustava korišten u vašim podacima za treniranje i unesite svoje testno pitanje. Primijetit ćete da su obje strane ažurirane s istim kontekstom i pitanjem. Pokrenite usporedbu i vidjet ćete razliku u izlazima između njih. _Primijetite kako fino podešeni model prikazuje odgovor u formatu koji ste dali u svojim primjerima, dok osnovni model jednostavno slijedi sistemski prompt_.

![Fine-tuning job status](../../../../../translated_images/hr/fine-tuned-playground-launch.5a26495c983c6350.webp)

Primijetit ćete da usporedba također prikazuje broj tokena za svaki model i vrijeme potrebno za izvođenje. **Ovaj specifični primjer je pojednostavljen i služi za prikaz procesa, ali ne odražava stvarni skup podataka ili scenarij**. Možda ćete primijetiti da oba uzorka prikazuju isti broj tokena (kontekst sustava i korisnički prompt su identični), pri čemu fino podešeni model troši više vremena za izvođenje (prilagođeni model).

U stvarnim scenarijima nećete koristiti ovakav jednostavan primjer, već fino podešavanje na stvarnim podacima (npr. katalog proizvoda za korisničku podršku) gdje će kvaliteta odgovora biti mnogo vidljivija. U _tom_ kontekstu, dobivanje ekvivalentne kvalitete odgovora s osnovnim modelom zahtijevat će više prilagođavanja prompta, što će povećati korištenje tokena i potencijalno vrijeme obrade za izvođenje. _Za isprobavanje, pogledajte primjere fino podešavanja u OpenAI Cookbooku za početak._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Odricanje od odgovornosti**:
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo postići točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za kritične informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
